In [1]:
import pandas as pd
import numpy as np
from sklearn import linear_model
from sklearn.model_selection import cross_val_score
from sklearn.model_selection import RepeatedKFold
from pygam import LinearGAM, s, f
from sklearn.model_selection import train_test_split
from sklearn.model_selection import GridSearchCV

from sklearn.pipeline import Pipeline
import feature_engine.imputation as imp
from feature_engine.encoding import OneHotEncoder

import matplotlib.pyplot as plt

In [5]:
lin_reg_low = pd.read_csv("lin_reg_analysis_low_tol.csv")
lin_reg_high = pd.read_csv("lin_reg_analysis_high_tol.csv")

In [8]:
lin_reg_low.sort_values("coefficients")

,variable,coefficients,intercept
9,Size_0.1-10MB,-2.320462e+07,2.451705e+07
18,Genres_Education,-1.561648e+07,1.663020e+07
6,Category_Rare,-1.524437e+07,1.674608e+07
4,Category_FAMILY,-1.279169e+07,1.814766e+07
12,Size_20.1-30MB,-1.214223e+07,1.714140e+07
17,Genres_Rare,-1.106786e+07,1.752285e+07
14,Content Rating_Everyone,-1.082337e+07,2.449181e+07
2,days_since_update,-8.727139e+06,1.581264e+07
10,Size_10.1-20MB,-8.311738e+06,1.711518e+07
16,Genres_Entertainment,-7.068245e+06,1.623403e+07


In [9]:
lin_reg_high.sort_values("coefficients")

,variable,coefficients,intercept
29,Size_0.1-10MB,-2.320462e+07,2.451705e+07
39,Type_Paid,-1.701914e+07,1.709222e+07
8,Category_MEDICAL,-1.638802e+07,1.650618e+07
49,Genres_Education,-1.561648e+07,1.663020e+07
44,Content Rating_Rare,-1.543213e+07,1.581976e+07
16,Category_Rare,-1.524437e+07,1.674608e+07
17,Category_DATING,-1.503149e+07,1.613851e+07
19,Category_LIFESTYLE,-1.485349e+07,1.632477e+07
6,Category_FINANCE,-1.425128e+07,1.626292e+07
20,Category_BUSINESS,-1.422507e+07,1.640973e+07
